In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV

from sklearn.neural_network import MLPRegressor

# Add the parent directory to the Python path to load funtions from file ML_funtions
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

# Import helperfunctions
from ML_functions import fun_load_data, fun_preprocessing, fun_fit_tuning, fun_load_best_params
from ML_functions import fun_convert_time
from ML_functions import fun_scaled_neg_MAPE, fun_tuning_results, fun_scores, fun_benchmark_evaluation

# Assign string "TSP" or "CVRP" to the following variable to define the optimization problem
optimization_problem = "BPP"

# Load data
data, start_script = fun_load_data(optimization_problem)

# Do the train test split during the preprocessing
X_train, X_test, y_train, y_test, train_data = fun_preprocessing(data, train_size=0.8)

# **Analyse the data set**

In [2]:
# View the data frame and the column names (the features and the target variable)
display(train_data, train_data.columns)

# Feature categories
instance_features = ["Number Items", "Item Weight Ratio", "Item Size Ratio", "Bin Weight", "Bin Size"]
weight_and_sum_features = ["Weight Size Sum Ratio", "Item Volume Ratio", "Item Density Ratio"]
utilization_features = ["Item Total Bin Utilization Ratio", "Final Total Bin Utilization"]
combination_features = ["Total Bin Combinations Ratio", "Perfect Total Bin Combinations Ratio"]
statistical_features = ["Weight Sum", "Size Sum", "Weight Std", "Size Std", "Weight Max", 
                        "Size Max", "Weight Min", "Size Min", "Correlation", "Skewness Weight", "Skewness Size"]
cost_features = ["Marginal Costs/Bins Ratio", "Total Bins"] #"Shapley Value"
 
# All features
all_features = instance_features + weight_and_sum_features + cost_features + statistical_features + utilization_features + combination_features
print(f"Number of features: {len(all_features)}")
print(f"Number of columns: {len(train_data.columns)}")
print(f"Number of rows (itmes): {len(train_data)}")

,,Number Items,Item Weight Ratio,Item Size Ratio,Bin Weight,Bin Size,Weight Size Sum Ratio,Item Volume Ratio,Item Density Ratio,Item Total Bin Utilization Ratio,Total Bin Combinations Ratio,...,Size Max,Weight Min,Size Min,Correlation,Skewness Weight,Skewness Size,Final Total Bin Utilization,Marginal Cost/Bins Ratio,Total Bins,Shapley Value
Index,Instance ID,,,,,,,,,,,,,,,,,,,,,
0,0,7,1.250000,1.272727,25,25,1.258427,1.555556,0.579950,1.258427,0.816667,...,10,6,1,0.197203,0.000000,0.561304,0.593333,1.00,3,0.480952
1,0,7,0.750000,1.060606,25,25,0.865169,0.777778,0.417564,0.865169,1.166667,...,10,6,1,0.197203,0.000000,0.561304,0.593333,1.00,3,0.380952
2,0,7,0.875000,0.848485,25,25,0.865169,0.725926,0.608948,0.865169,1.050000,...,10,6,1,0.197203,0.000000,0.561304,0.593333,1.00,3,0.414286
3,0,7,1.125000,2.121212,25,25,1.494382,2.333333,0.313173,1.494382,0.933333,...,10,6,1,0.197203,0.000000,0.561304,0.593333,1.00,3,0.447619
4,0,7,1.250000,0.212121,25,25,0.865169,0.259259,3.479702,0.865169,0.816667,...,10,6,1,0.197203,0.000000,0.561304,0.593333,1.00,3,0.480952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296995,26999,15,0.947368,0.600000,23,19,0.794118,0.583153,0.982354,0.794118,1.260442,...,10,2,1,-0.109569,-0.384261,0.460115,0.807780,1.25,5,0.297061
296996,26999,15,1.578947,1.000000,23,19,1.323529,1.619870,0.982354,1.323529,0.597052,...,10,2,1,-0.109569,-0.384261,0.460115,0.807780,1.25,5,0.430081
296997,26999,15,0.315789,1.000000,23,19,0.617647,0.323974,0.196471,0.617647,1.488943,...,10,2,1,-0.109569,-0.384261,0.460115,0.807780,0.00,5,0.189644


Index(['Number Items', 'Item Weight Ratio', 'Item Size Ratio', 'Bin Weight',
       'Bin Size', 'Weight Size Sum Ratio', 'Item Volume Ratio',
       'Item Density Ratio', 'Item Total Bin Utilization Ratio',
       'Total Bin Combinations Ratio', 'Perfect Total Bin Combinations Ratio',
       'Weight Sum', 'Size Sum', 'Weight Std', 'Size Std', 'Weight Max',
       'Size Max', 'Weight Min', 'Size Min', 'Correlation', 'Skewness Weight',
       'Skewness Size', 'Final Total Bin Utilization',
       'Marginal Cost/Bins Ratio', 'Total Bins', 'Shapley Value'],
      dtype='object')

Number of features: 25
Number of columns: 26
Number of rows (itmes): 297000


In [3]:
# Get the min/max of instance size
min_instance_size = min(train_data["Number Items"])
max_instance_size = max(train_data["Number Items"])
number_of_instances_per_size = int((max(train_data.index.get_level_values(level="Instance ID")) + 1) / len(np.unique(train_data["Number Items"])))
print(f"Min/Max instance size: ({min_instance_size}, {max_instance_size})")
print(f"Number of instances per size: {number_of_instances_per_size}")

# Get the min/max of item weight and size
min_item_weight = min(data["Item Weight"])
max_item_weight = max(data["Item Weight"])
min_item_size = min(data["Item Size"])
max_item_size = max(data["Item Size"])
print(f"Min/Max item weight: ({min_item_weight}, {max_item_weight})")
print(f"Min/Max item size: ({min_item_size}, {max_item_size})")

# Get the min/max of bin weight and size
min_bin_weight = min(train_data["Bin Weight"])
max_bin_weight = max(train_data["Bin Weight"])
min_bin_size = min(train_data["Bin Size"])
max_bin_size = max(train_data["Bin Size"])
print(f"Min/Max bin weight: ({min_bin_weight}, {max_bin_weight})")
print(f"Min/Max bin size: ({min_bin_size}, {max_bin_size})")

Min/Max instance size: (7, 15)
Number of instances per size: 3000
Min/Max item weight: (1, 10)
Min/Max item size: (1, 10)
Min/Max bin weight: (17, 25)
Min/Max bin size: (17, 25)


# **Neural Network - Multi Layer Perceptron**
**Hyperparametertuning**

In [4]:
# Define a pipeline
pipe = make_pipeline(StandardScaler(), 
                     MLPRegressor(hidden_layer_sizes=(64, 32, 16), activation="relu", learning_rate="adaptive",
                                  max_iter=1000, random_state=42))

# Define parameter grid
param_grid = {"mlpregressor__solver": ["sgd", "adam"],
              "mlpregressor__alpha": [0.0001, 0.001, 0.01],
              "mlpregressor__batch_size": [32, 64, 128], 
              "mlpregressor__learning_rate_init": [0.0001, 0.001], 
              "mlpregressor__early_stopping": [True, False]}

# Grid search
grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, 
                           cv=GroupKFold(n_splits=3).split(X_train, y_train, groups=X_train.index.get_level_values(level="Instance ID")), 
                           scoring=fun_scaled_neg_MAPE, refit=False, verbose=True, n_jobs=-1)
tuning_details = fun_fit_tuning(grid_search, X_train, y_train, file_name=f"{optimization_problem}_NN")

# Estimate model performance with cross validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(grid_search, X_train, y_train)
model_results_dict.update(tuning_details)

# View grid search CV scores of all parameter combinations
results_df = fun_tuning_results(grid_search, param_grid)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


{'Search type': 'GridSearchCV',
 'Parameter combinations': 72,
 'Total tuning time': '1h, 15m',
 'Total tuning fit time': '4h, 57m',
 'Total tuning prediction time': '1m, 34s'}

CV MAPE (scaled) train data: 3.42 %


**Best model / parameter combination:**

{'mlpregressor__alpha': 0.0001,
 'mlpregressor__batch_size': 64,
 'mlpregressor__early_stopping': True,
 'mlpregressor__learning_rate_init': 0.0001,
 'mlpregressor__solver': 'adam'}

**Cross validation scores of different parameter combinations:**

,solver,alpha,batch_size,learning_rate_init,early_stopping,mean_test_score,converted_mean_fit_time
0,adam,0.0001,64,0.0001,True,-0.034177,"3m, 9s"
1,adam,0.0001,32,0.0001,True,-0.034368,"5m, 11s"
2,adam,0.0001,128,0.0010,True,-0.034403,"1m, 6s"
3,adam,0.0010,128,0.0001,True,-0.035006,"3m, 39s"
4,adam,0.0001,64,0.0010,True,-0.035255,"2m, 45s"
5,adam,0.0001,128,0.0001,True,-0.035325,"2m, 30s"
6,adam,0.0010,64,0.0001,True,-0.035793,"3m, 31s"
7,adam,0.0010,128,0.0010,True,-0.036144,"2m, 38s"
8,adam,0.0001,64,0.0010,False,-0.036638,31s
9,adam,0.0001,128,0.0010,False,-0.036808,21s


,solver,alpha,batch_size,learning_rate_init,early_stopping,mean_test_score,converted_mean_fit_time
60,sgd,0.0100,128,0.0010,False,-0.098449,"1m, 12s"
61,sgd,0.0010,128,0.0010,False,-0.100040,"1m, 28s"
62,sgd,0.0001,128,0.0010,False,-0.100114,"1m, 36s"
63,sgd,0.0100,32,0.0001,False,-0.110260,"2m, 32s"
64,sgd,0.0010,32,0.0001,False,-0.116812,"2m, 24s"
65,sgd,0.0001,32,0.0001,False,-0.117004,"2m, 27s"
66,sgd,0.0100,64,0.0001,False,-0.132149,"1m, 37s"
67,sgd,0.0010,64,0.0001,False,-0.132959,"1m, 42s"
68,sgd,0.0001,64,0.0001,False,-0.133041,"2m, 0s"
69,sgd,0.0100,128,0.0001,False,-0.151641,"1m, 8s"


**Test Score**

In [5]:
# Load best parameters of the model
best_params = fun_load_best_params(optimization_problem, model_abbreviation="NN")

# Create pipeline
pipe = make_pipeline(StandardScaler(), 
                     MLPRegressor(hidden_layer_sizes=(256, 128, 64), activation="relu", learning_rate="adaptive",
                                  max_iter=1000, random_state=42))
pipe.set_params(**best_params)

# Estimate model performance with cross-validation on the train set and get scores on test set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(pipe, X_train, y_train, X_test, y_test, compute_test_scores=True)

# Create a dictionary to store the results
results_dict = {"Neural Network": model_results_dict}

{'mlpregressor__alpha': 0.0001,
 'mlpregressor__batch_size': 64,
 'mlpregressor__early_stopping': True,
 'mlpregressor__learning_rate_init': 0.0001,
 'mlpregressor__solver': 'adam'}

CV MAPE (scaled) train data: 3.19 %
CV RMSE (scaled) train data: 0.01
CV computation time: 7m, 48s

MAPE (scaled) test data: 3.0300000000000002 %
RMSE (scaled) test data: 0.01
Model fit time: 11m, 35s
Model prediction time: 0s


**MAPE and RMSE on test data per instance size:**

Number Items,7,8,9,10,11,12,13,14,15,Mean
MAPE,3.09,3.30,3.43,3.25,3.09,2.89,3.08,2.76,2.76,3.03
RMSE,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01


### **Evaluation of the *Share of Weight and Size* benchmark**

In [6]:
# Compute benchmark "Share of Weight and Size"
# Multiply the the number of bins with half of the sum of the customers' weight and size shares in the instance
columns = ["Instance ID", "Number Items", "Item Weight", "Item Size", "Weight Sum", "Size Sum", "Total Bins", "Shapley Value"]
data_set = data[columns].copy()
data_set["Φ SWS"] = data_set.groupby(by="Instance ID").apply(
    lambda group: group["Total Bins"] * (0.5 * (group["Item Weight"]/group["Weight Sum"]) + 
                                         0.5 * (group["Item Size"]/group["Size Sum"]))).reset_index(drop=True)
display(data_set.head(7))

# Do the train test split during the preprocessing
X_train, X_test, y_train, y_test, train_data = fun_preprocessing(data_set, train_size=0.8)

,Instance ID,Number Items,Item Weight,Item Size,Weight Sum,Size Sum,Total Bins,Shapley Value,Φ SWS
0,0,7,10,6,56,33,3,0.480952,0.540584
1,0,7,6,5,56,33,3,0.380952,0.387987
2,0,7,7,4,56,33,3,0.414286,0.369318
3,0,7,9,10,56,33,3,0.447619,0.695617
4,0,7,10,1,56,33,3,0.480952,0.313312
5,0,7,6,2,56,33,3,0.380952,0.251623
6,0,7,8,5,56,33,3,0.414286,0.441558


In [7]:
# Evaluate the "Weight Size" benchmark
results_dict = fun_benchmark_evaluation(X_train, X_test, y_train, y_test, benchmark_str="Φ SWS", results_dict=results_dict)

,Train set,Test set
MAPE,22.21,22.00
RMSE,0.09,0.09


Number Items,7,8,9,10,11,12,13,14,15,Mean
MAPE,23.49,22.62,22.39,21.17,21.68,22.01,21.97,21.83,21.72,22.00
RMSE,0.10,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.08,0.09


### **Store the results**

In [8]:
# Create run times Data Frame
run_times_df = pd.DataFrame({key: [model_results_dict[key]] for key in ["CV computation time", "Model fit time", "Model prediction time"]})

# Get scores of the neural network
mlp_scores = pd.DataFrame(data=[results_dict["Neural Network"]["MAPE"].values(), 
                                results_dict["Neural Network"]["RMSE"].values()], 
                          columns=["Train set", "Test set"], index=["MAPE", "RMSE"])
mlp_scores.columns.name = "MLP"
mlp_cat_scores = results_dict["Neural Network"]["Scores per instance size"]
mlp_cat_scores.columns.name = "MLP / No. Items"

# Get scores of the WS benchmark
ws_scores = results_dict["Φ SWS"][0]
ws_scores.columns.name = "Φ SWS"
ws_cat_scores = results_dict["Φ SWS"][1]
ws_cat_scores.columns.name = "Φ SWS / No. Items"
display(run_times_df, mlp_scores, ws_scores, mlp_cat_scores, ws_cat_scores)

# Save data frames with results into an excel file
file_path = "04_test_results/" + optimization_problem + "_results.xlsx"

# Use ExcelWriter to write multiple DataFrames to the same file
with pd.ExcelWriter(file_path) as writer:
    run_times_df.to_excel(writer, sheet_name="run_times")
    mlp_scores.to_excel(writer, sheet_name="mlp_scores")
    ws_scores.to_excel(writer, sheet_name="ws_scores")
    mlp_cat_scores.to_excel(writer, sheet_name="mlp_cat_scores")
    ws_cat_scores.to_excel(writer, sheet_name="ws_cat_scores")
print("File saved succesfully.")

# Print total script run time
print("\nTotal script computation time:", fun_convert_time(start=start_script, end=time.time()))

,CV computation time,Model fit time,Model prediction time
0,"7m, 48s","11m, 35s",0s


MLP,Train set,Test set
MAPE,3.19,3.03
RMSE,0.01,0.01


Φ SWS,Train set,Test set
MAPE,22.21,22.00
RMSE,0.09,0.09


MLP / No. Items,7,8,9,10,11,12,13,14,15,Mean
MAPE,3.09,3.30,3.43,3.25,3.09,2.89,3.08,2.76,2.76,3.03
RMSE,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01


Φ SWS / No. Items,7,8,9,10,11,12,13,14,15,Mean
MAPE,23.49,22.62,22.39,21.17,21.68,22.01,21.97,21.83,21.72,22.00
RMSE,0.10,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.08,0.09


File saved succesfully.

Total script computation time: 1h, 37m
